# Time Series GIF from GEO Example

This example shows how the Capella API can be used to fetch a time series stack of data, read data for a given bounding box directly from cloud optimized geotiffs stored in Capella's S3 bucket, and create a time series gif for visualization. To run this notebook, you will need a Capella API account, with credentials saved in a credentials.json file.

In [ ]:
import requests
import json
import rasterio
import cv2
import imageio as io # requires Python 3.5+ see http://imageio.github.io/ for more info
import numpy as np
from IPython.display import HTML
from pyproj import Transformer
from rasterio.windows import Window
from skimage import exposure
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance

### Set up project variables

In [ ]:
# Data, BBOX, and time definition
bbox = [4.3554, 51.9015, 4.3662, 51.9089]
timerange = "2019-08-01T00:00:00Z/2019-08-30T23:59:00Z"
collections = ["rotterdam-aerial-mosaic"]

# Windows sizes for filtering
FILTSIZE = 3 # window size for speckle filter

### Function to apply speckle filtering

In [ ]:
def lee_filter(img, size):
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

### Function to Apply Linear Stretch

In [ ]:
# This performs a linear stretch clipping the lower and upper percent specified
def stretch(bands, lower_percent=1, higher_percent=98): # adjust lower and upper percent values for best visualization
    import numpy as np
    np.ma.array(bands, mask=np.isnan(bands))
    out = np.zeros_like(bands)
    a = 0 
    b = 255 
    c = np.percentile(bands, lower_percent)
    d = np.percentile(bands, higher_percent)        
    t = a + (bands - c) * (b - a) / (d - c)    
    t[t<a] = a
    t[t>b] = b
    out = t
    return out.astype(np.uint8)

### Function to reproject bounding box coordinates to the SAR data coordinate system

In [ ]:
def reproject_bbox(bb, epsg):
    transformer = Transformer.from_crs("EPSG:4326", epsg, always_xy=True)
    obb = [0] * 4
    obb[0], obb[1] = transformer.transform(bb[0], bb[1])
    obb[2], obb[3] = transformer.transform(bb[2], bb[3])
    return obb   

### Load API credentials and get an API token

In [ ]:
# Load username and password
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

In [ ]:
# Get token
r = requests.post('https://api.capellaspace.com/token', 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))
accesstoken = r.json()["accessToken"]
headers = {'Authorization':'Bearer ' + accesstoken}

### Use the API to search for Capella SAR data

In [ ]:
# Post search filters
filters = {
    "datetime": timerange,
    "bbox": bbox,
    "collections": collections,
    "query": {"sar:instrument_mode": {"eq": "stripmap"},  
            "sar:product_type": {"eq":"GEO"}},
    "sort": [{"field": "dtr:start_datetime","direction": "asc"}],
    "limit": 50
}

headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}
r = requests.post('https://api.capellaspace.com/catalog/search', json=filters, headers=headers)

In [ ]:
# Build the Order
features = r.json()["features"]
granulelist = []

# Loop over all the features from the response and add to an array for an order
for f in features:
    item = {"CollectionId": f["collection"], "GranuleId": f["id"]}
    granulelist.append(item)
    
myorder = {"Items": granulelist}
#print(myorder)
# Post the order and inspect the result
r = requests.post('https://api.capellaspace.com/orders', json=myorder, headers=headers)

In [ ]:
myorderid = r.json()["orderId"]
r = requests.get('https://api.capellaspace.com/orders/' + myorderid + '/download', headers=headers)

### Build a time series GIF from the time series

Ingests the stack of images ordered from the API and assembles a time series GIF

In [ ]:
#Sort the results in time
features = r.json()
assetlist = {}
for f in features:
    assetlist.update( {f["properties"]["datetime"] : f["assets"]["HH"]["href"]})   

In [ ]:
timeseries = []
# Open the file with Rasterio
Session = rasterio.Env(
            GDAL_DISABLE_READDIR_ON_OPEN='YES',
            CPL_VSIL_CURL_USE_HEAD='NO',
            )

for key in sorted(assetlist.keys()):   
    with Session:
        filepath = assetlist[key]
        timestamp = key
        with rasterio.open(filepath) as src:
            meta = src.meta
            rbbox = reproject_bbox(bbox, src.crs)
            w = rasterio.windows.from_bounds(rbbox[0], rbbox[1], rbbox[2], rbbox[3], src.transform)
            img1 = src.read(1, window=w)
            img1[img1 == meta['nodata']] = 0
            img1 = lee_filter(img1, FILTSIZE)
            img1 = exposure.adjust_log(img1, gain=10)
            
            img1_scaled = stretch(img1) 

            img1_rgb = np.zeros((img1_scaled.shape[0],img1_scaled.shape[1],3), np.uint8)
            img1_rgb[:,:,0] = img1_scaled
            img1_rgb[:,:,1] = img1_scaled
            img1_rgb[:,:,2] = img1_scaled
            
            img1_rgb = cv2.putText(img1_rgb, text=timestamp, org=(10,30),fontFace=2, fontScale=1, color=(0,255,255), thickness=2)
            
            timeseries.append(img1_rgb)

io.mimsave('timeseries.gif', timeseries, fps=1)

In [ ]:
HTML('<img src="timeseries.gif">')